In [1]:
import pandas as pd
import os
from typing import List, Sequence, Optional
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from typing import Mapping, Hashable, Optional, Iterable
from plotly.subplots import make_subplots
from statsmodels.formula.api import ols
import re
from backend.cuentas_nacionales.pib import PIBViews
from backend.cuentas_nacionales.pib.pib_constantes import ConstantesPIB as C
from backend.political_terms import tag_politics
from backend.plots import (plot_timeseries_bars, plot_bar_subplots_by_administration, 
                           simple_linegraph, plot_bars, create_pie_chart, 
                           plot_contributions)
from backend.utils import compute_basic_statistics, calculate_component_percentages, validate_percentage_sum, aggregate_columns
from backend.contributions import compute_contributions_pipeline, summary_contributions

In [2]:
g = lambda x: x.map(lambda x:x if type(x) == str else '{:,.2f}'.format(x))

year_on_year_changes = lambda df, q: (df.pct_change(q) * 100).dropna()

In [3]:
# pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [4]:
# colores por presidente:

presidential_colors = {
    "Calderón":   "#9ecae1",  
    "Olsen":      "#fb6a4a",  
    "Rodríguez":  "#fdd0a2",  
    "Pacheco":    "#9467bd",  
    "Arias":      "#74c476",  
    "Chinchilla": "#fdae6b",  
    "Solís":      "#c7c7c7",  
    "Alvarado":   "#e377c2",  
    "Chaves":     "#6baed6",  
}

# <font color = 'red'> 1. DATOS

In [5]:
folder = r'C:\Users\adolj\OneDrive\Documentos\APPS\informed_economist\data\raw'
file_name = 'Variables_PIB_TC.xlsx'
input_path = os.path.join(folder, file_name)
quarterly_data_levels_TC = pd.read_excel(input_path, index_col= 'fecha')

# Datos en millones de colones
quarterly_data_levels_TC.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 139 entries, 1991-03-31 to 2025-09-30
Data columns (total 45 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   PIB_TC                                       139 non-null    float64
 1   PIB_Impuestos_TC                             139 non-null    float64
 2   PIB_Valor_Agregado_TC                        139 non-null    float64
 3   PIB_Agricultura_Silvicultura_Pesca_TC        139 non-null    float64
 4   PIB_Minas_Canteras_TC                        139 non-null    float64
 5   PIB_Manufactura_TC                           139 non-null    float64
 6   PIB_Electricidad_Agua_Saneamiento_TC         139 non-null    float64
 7   PIB_Construccion_TC                          139 non-null    float64
 8   PIB_Comercio_TC                              139 non-null    float64
 9   PIB_Transporte_Almacenamiento_TC             139 non-null

In [6]:
opibv = PIBViews(quarterly_data_levels_TC)

## <font color = 'red'> 1.1 DATOS EN TASAS DE CRECIMIENTO INTERANUAL

### <font color = 'red'> 1.1 TOTAL

In [7]:
# datos en millones de colones
pib = opibv.oferta.get_pib()
# datos en tasas de crecimiento interanuales
pib = year_on_year_changes(pib, q = 4)
# etiqueta de administración
pib = tag_politics(pib)

display(pib)

pib.to_csv(r'C:\Users\adolj\OneDrive\Documentos\APPS\governomics\governomicsCR\informed_economist\data\datasets\pib_yoy.txt')

,PIB_TC,President,Party,Term,Label
fecha,,,,,
1992-03-31,8.218815,Rafael Ángel Calderón Fournier,PUSC,1990–1994,Calderón
1992-06-30,9.265592,Rafael Ángel Calderón Fournier,PUSC,1990–1994,Calderón
1992-09-30,9.733945,Rafael Ángel Calderón Fournier,PUSC,1990–1994,Calderón
1992-12-31,8.975770,Rafael Ángel Calderón Fournier,PUSC,1990–1994,Calderón
1993-03-31,8.043025,Rafael Ángel Calderón Fournier,PUSC,1990–1994,Calderón
...,...,...,...,...,...
2024-09-30,4.245729,Rodrigo Chaves Robles,PPSD,2022–2026,Chaves
2024-12-31,4.329637,Rodrigo Chaves Robles,PPSD,2022–2026,Chaves
2025-03-31,4.113338,Rodrigo Chaves Robles,PPSD,2022–2026,Chaves


### <font color = 'red'> 1.2 OFERTA: ACTIVIDAD ECONÓMICA

In [8]:
# datos en millones de colones
pib_ind = opibv.oferta.get_industrias()
# datos en tasas de crecimiento interanuales
pib_ind = year_on_year_changes(pib_ind, q = 4)
# etiqueta de administración
pib_ind = tag_politics(pib_ind)

display(g(pib_ind.tail(3)))

pib_ind.to_csv(r'C:\Users\adolj\OneDrive\Documentos\APPS\governomics\governomicsCR\informed_economist\data\datasets\pib_yoy_industrias.txt')

,PIB_Agricultura_Silvicultura_Pesca_TC,PIB_Minas_Canteras_TC,PIB_Manufactura_TC,PIB_Electricidad_Agua_Saneamiento_TC,PIB_Construccion_TC,PIB_Comercio_TC,PIB_Transporte_Almacenamiento_TC,PIB_Hoteles_Restaurantes_TC,PIB_Informacion_Comunicaciones_TC,PIB_Financieras_Seguros_TC,PIB_Inmobiliario_TC,PIB_Actividades_Profesionales_TC,PIB_Administracion_Publica_TC,PIB_Ense_Salud_Asistencia_Social_TC,PIB_Otras_Actividades_TC,President,Party,Term,Label
fecha,,,,,,,,,,,,,,,,,,,
2025-03-31,-2.93,1.33,8.49,3.69,1.89,2.47,6.82,-0.09,3.91,6.53,4.74,5.75,1.47,2.14,1.76,Rodrigo Chaves Robles,PPSD,2022–2026,Chaves
2025-06-30,-2.60,2.71,11.19,5.47,-1.06,2.70,6.22,1.68,3.62,5.90,4.42,5.68,1.27,2.43,1.60,Rodrigo Chaves Robles,PPSD,2022–2026,Chaves
2025-09-30,-1.92,3.32,12.80,8.87,-2.76,3.15,5.64,2.66,4.08,5.99,4.11,6.76,0.82,2.49,1.80,Rodrigo Chaves Robles,PPSD,2022–2026,Chaves


### <font color = 'red'> 1.3 OFERTA: SECTORES

In [9]:
# datos en millones de colones
pib_sectores = opibv.oferta.get_sectores()
# datos en tasas de crecimiento interanuales
pib_sectores = year_on_year_changes(pib_sectores, q = 4)
# etiqueta de administración
pib_sectores = tag_politics(pib_sectores)

display(g(pib_sectores))

pib_sectores.to_csv(r'C:\Users\adolj\OneDrive\Documentos\APPS\governomics\governomicsCR\informed_economist\data\datasets\pib_yoy_sectores.txt')

,Agro_TC,Servicios_TC,Industria_Ampliada_TC,President,Party,Term,Label
fecha,,,,,,,
1992-03-31,5.24,5.08,11.92,Rafael Ángel Calderón Fournier,PUSC,1990–1994,Calderón
1992-06-30,8.91,6.13,12.67,Rafael Ángel Calderón Fournier,PUSC,1990–1994,Calderón
1992-09-30,7.23,7.04,12.31,Rafael Ángel Calderón Fournier,PUSC,1990–1994,Calderón
1992-12-31,2.32,7.22,8.74,Rafael Ángel Calderón Fournier,PUSC,1990–1994,Calderón
1993-03-31,1.83,6.57,7.15,Rafael Ángel Calderón Fournier,PUSC,1990–1994,Calderón
...,...,...,...,...,...,...,...
2024-09-30,2.62,4.38,4.03,Rodrigo Chaves Robles,PPSD,2022–2026,Chaves
2024-12-31,-0.51,4.05,5.24,Rodrigo Chaves Robles,PPSD,2022–2026,Chaves
2025-03-31,-2.93,3.86,6.63,Rodrigo Chaves Robles,PPSD,2022–2026,Chaves


### <font color = 'red'> 1.4 DEMANDA: COMPONENTES DEL GASTO

In [10]:
# datos en millones de colones
pib_comp_gasto = opibv.demanda.get_categorias_principales()
# datos en tasas de crecimiento interanuales
pib_comp_gasto = year_on_year_changes(pib_comp_gasto, q = 4)
# etiqueta de administración
pib_comp_gasto = tag_politics(pib_comp_gasto)

pib_comp_gasto.drop('PIB_TC', axis = 1, inplace = True)

display(g(pib_comp_gasto.tail(3)))

pib_comp_gasto.to_csv(r'C:\Users\adolj\OneDrive\Documentos\APPS\governomics\governomicsCR\informed_economist\data\datasets\pib_yoy_componentes_gasto.txt')

,PIB_Gasto_Consumo_Final_Hogares_TC,PIB_Gasto_Consumo_Final_Gobierno_General_TC,PIB_Formacion_Bruta_Capital_Fijo_TC,PIB_Exportaciones_Bienes_Servicios_TC,PIB_Importaciones_Bienes_Servicios_TC,President,Party,Term,Label
fecha,,,,,,,,,
2025-03-31,3.08,2.19,6.96,6.97,3.89,Rodrigo Chaves Robles,PPSD,2022–2026,Chaves
2025-06-30,2.77,2.28,3.18,8.39,4.91,Rodrigo Chaves Robles,PPSD,2022–2026,Chaves
2025-09-30,3.67,2.12,2.24,9.88,7.65,Rodrigo Chaves Robles,PPSD,2022–2026,Chaves


### <font color = 'red'> 1.4 RÉGIMEN DE COMERCIO (RÉGIMEN DEFINITIVO Y ZONAS FRANCAS)

In [11]:
# datos en millones de colones
pib_regimen = opibv.dataframe[['PIB_RegDef_TC', 'PIB_RegEsp_TC']]
# datos en tasas de crecimiento interanuales
pib_regimen = year_on_year_changes(pib_regimen, q = 4)
# etiqueta de administración
pib_regimen = tag_politics(pib_regimen)

display(g(pib_regimen))

pib_regimen.to_csv(r'C:\Users\adolj\OneDrive\Documentos\APPS\governomics\governomicsCR\informed_economist\data\datasets\pib_yoy_regimen.txt')

,PIB_RegDef_TC,PIB_RegEsp_TC,President,Party,Term,Label
fecha,,,,,,
1992-03-31,7.81,25.63,Rafael Ángel Calderón Fournier,PUSC,1990–1994,Calderón
1992-06-30,8.96,24.30,Rafael Ángel Calderón Fournier,PUSC,1990–1994,Calderón
1992-09-30,9.50,25.83,Rafael Ángel Calderón Fournier,PUSC,1990–1994,Calderón
1992-12-31,8.74,27.44,Rafael Ángel Calderón Fournier,PUSC,1990–1994,Calderón
1993-03-31,7.81,26.76,Rafael Ángel Calderón Fournier,PUSC,1990–1994,Calderón
...,...,...,...,...,...,...
2024-09-30,3.59,8.20,Rodrigo Chaves Robles,PPSD,2022–2026,Chaves
2024-12-31,3.53,9.88,Rodrigo Chaves Robles,PPSD,2022–2026,Chaves
2025-03-31,3.22,10.04,Rodrigo Chaves Robles,PPSD,2022–2026,Chaves
